In [154]:
import os
import pandas as pd
import matplotlib
import numpy as np
pd.set_option('display.max_colwidth', None)

In [7]:
path = "data/t_ser_snr/"
dict_t = {}
for i, file in enumerate(os.listdir(path)):
    df = pd.read_csv(path+file)
    df = df[["MAC_masked", "name_ap"]]
    dict_t[i] = df.to_dict()

In [161]:
def room_division(dataRoom):
    """
    room_division
    -------------
    take all the data and divide the rooms of all PoliTo
    ### Output:
        - the dataframe with the room divided
    """
    Ap = "AP-AULA"
    dataRoom = dataRoom[dataRoom["name_ap"].str.contains(Ap)]
    rooms = pd.DataFrame()
    rooms[["AP","Room", "APnum", "NaN"]] = dataRoom["name_ap"].str.split(
        "-", expand=True
    )
    dataRoom = pd.concat([dataRoom, rooms], axis=1)
    dataRoom = dataRoom.drop(["AP", "NaN"], axis=1)
    return dataRoom

In [162]:
df_1 = room_division(pd.DataFrame.from_dict(dict_t[0]))
df_2 = room_division(pd.DataFrame.from_dict(dict_t[2]))

In [163]:
mac_giova ="34a9dea164698cb433dcf4b20943976fc133db4c927005ffc340c4ec380fb18d"

In [166]:
df_moved = df_1.merge(df_2, on=['MAC_masked'], how="inner",indicator=True)

In [167]:
df_moved = df_moved.drop(["name_ap_x", "name_ap_y", "APnum_x", "APnum_y", "_merge"], axis=1)

In [173]:
OD_matr =df_moved.assign(count=1).pivot_table(index="Room_x", columns="Room_y", values="count", aggfunc="count").fillna(0)

In [174]:
OD_matr.to_csv("od.csv")